In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pickle
import numpy as np

USE_SUBLABEL = False
URL_PER_SITE = 10
TOTAL_URLS   = 950

# Load the pickle file
print("Loading datafile...")
with open("/content/drive/MyDrive/mon_standard.pkl", 'rb') as fi: #Colab 에서 mon_standard.pkl 파일의 경로
    data = pickle.load(fi)

# X1 ~ X4: Continuous features(4개)

X4 = [] # Array to store cumulative packet sizes - 19,000 instances

#X5 ~ X17: Categorical features(13개)
         # Arrays to store ~
X5 = []  # rank 1.Number of incoming packets
X6 = []  # rank 2.Number of outgoing packets as a fraction of the total number of packets
X7 = []  # rank 3.Number of incoming packets as a fraction of the total number of packets

y = []  # Array to store the site of each instance - 19,000 instances, e.g., [0, 0, 0, 0, 0, 0, ..., 94, 94, 94, 94, 94]


# Differentiate instances and sites, and store them in the respective x and y arrays
# x array (direction*timestamp), y array (site label)
for i in range(TOTAL_URLS):
    if USE_SUBLABEL:
        label = i
    else:
        label = i // URL_PER_SITE # Calculate which site's URL the current URL being processed belongs to and set that value as the label. Thus, URLs fetched from the same site are labeled identically.

    for sample in data[i]:
        size_seq = [] #X2
        size_cumulative_seq = [] #X4
        outgoing_count = 0
        incoming_count = 0 #X5
        outgoing_fraction = 0.0 #X6
        incoming_fraction = 0.0 #X7

        last_sign = None
        cumulative_sum = 0

        for c in sample:

            # Calculate the direction and size for packet size
            dr = 1 if c > 0 else -1
            packet_size = dr * 512

           # X3
           # Calculate the burst sequence based on the sign of the value
            if c > 0:
                sign = 1
                outgoing_count += 1 #X6 (rank 1)

            else:
                sign = -1
                incoming_count += 1 #X5 (rank 1)

            last_sign = sign

            #X4
            cumulative_sum += packet_size
            size_cumulative_seq.append(cumulative_sum)


        #X6 (rank 2), X7 (rank 3)
        total_packets = len(sample)
        if total_packets > 0:
            outgoing_fraction = outgoing_count / total_packets #X6
            incoming_fraction = incoming_count / total_packets #X7



        X4.append(size_cumulative_seq)
        X5.append(incoming_count)
        X6.append(outgoing_fraction)
        X7.append(incoming_fraction)

        y.append(label)

X4_np = np.array([np.array(xi) for xi in X4])
X5_np = np.array([np.array(xi) for xi in X5])
X6_np = np.array([np.array(xi) for xi in X6])
X7_np = np.array([np.array(xi) for xi in X7])

size = len(y)

# Extract y values and remove duplicates
unique_y = list(set(y))

y_np = np.array(y)


np.savez('/content/drive/MyDrive/data_2_np_array.npz', X4=X4_np, X5=X5_np, X6=X6_np, X7=X7_np, y=y_np)

